# CNN for post-processing
The idea is to create a stack of neural networks. Where the second one takes the predictions from the first one as inputs. This second CNN should clean up the prediction e.g. remove noise and close roads. This is possible with a CNN because of the relation between adjecent labels

In [21]:
from dataset_preprocessing import create_dataset, extract_patches, compute_input_features, compute_output_features,load_image,img_crop_translate
from dataset_postprocessing import unpatch
from feature_extractors import extract_features, extract_features_2d, value_to_class, value_to_2d_class
from visualization_helpers import *

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as n

%matplotlib inline

In [22]:
# GLOBAL VARIABLES
ROOT_DIR = "post/"
TOTAL_IMAGES = 100 # Number of images to load
TRAIN_FRACTION = 0.8 # Percentage of images used for training
PATCH_SIZE = 16
PATCH_TRANSLATION = 0
FOREGROUND_THRESHOLD = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
ORIGINAL_IMAGE_WIDTH = 400
ORIGINAL_IMAGE_HEIGHT = 400
NUM_CHANNELS = 3
THEANO = False

In [23]:
def dont_extract(input):
    return input

In [24]:
dataset = create_dataset(ROOT_DIR, TOTAL_IMAGES, TRAIN_FRACTION)
patches = extract_patches(dataset, PATCH_SIZE, PATCH_TRANSLATION)

input_features = compute_input_features(patches[0:2], dont_extract) # train_img_patches and test_img_patches
output_features = compute_output_features(patches[2:4], value_to_2d_class, FOREGROUND_THRESHOLD) # train_gt_img_patches and test_gt_img_patches

from skimage import img_as_float
    
X_train = img_as_float(input_features[0])
Y_train = output_features[0].astype(np.float32)
X_test = img_as_float(input_features[1])
Y_test = output_features[1].astype(np.float32)

Original loaded dataset size: 20
Creating train dataset...
Skipping rotation.
Skipping flip.
Creating test dataset...
Skipping rotation.
Skipping flip.
Created train dataset size: 16
Created test dataset size: 4
Train patches: 10000
Test patches: 2500
Train GT patches: 10000
Test GT patches: 2500
Train features: 10000
Test features: 2500
Train GT features: 10000
Test GT features: 2500


In [25]:
from __future__ import print_function
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model
from keras.optimizers import SGD
from keras import backend as K

batch_size = 128
nb_classes = 2


# ********** Tuning parameters: (See Network architecture as well)

# Epochs to be trained
nb_epoch = 3
# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (4, 3)

"""
if "image_dim_ordering": is "th" and "backend": "theano", your input_shape must be (channels, height, width)
if "image_dim_ordering": is "tf" and "backend": "tensorflow", your input_shape must be (height, width, channels)
"""
if THEANO:
    input_shape = (NUM_CHANNELS, PATCH_SIZE, PATCH_SIZE)
else:
    input_shape = (PATCH_SIZE, PATCH_SIZE, NUM_CHANNELS)

def train_cnn(model_name='test.h5'):


    # **************** DEFINE THE MODEL ARCHITECTURE *******************

    model = Sequential()

    # Convolution layer with rectified linear activation
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],border_mode='same',input_shape=input_shape))
    model.add(Activation('relu'))

    # Second convolution
    model.add(Convolution2D(nb_filters, kernel_size[1], kernel_size[0]))
    model.add(Activation('relu'))

    model.add(Dropout(0.25))

    # Third convolution
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[0]))
    model.add(Activation('relu'))

    # Pooling and dropout
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    # Full-connected layer
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))

    model.add(Dense(1024))
    model.add(Activation('relu'))

    # Dropout to avoid overfitting
    model.add(Dropout(0.25))

    model.add(Dense(1024))
    model.add(Activation('relu'))

    # Dropout to avoid overfitting
    model.add(Dropout(0.5))

    #Fully-connected layer to ouptut the resulting class
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['fmeasure'])

    print(X_train.shape)
    #class_weight = auto??
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, class_weight='auto', verbose=1, validation_data=(X_test, Y_test))


    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

    model.save('models/' + model_name)
    
    data_dir = 'post/images/'
    pred_dir = 'post/prediction/'
    for i in range(0, 19):
        imageid = "pred_%.1d" % i
        image_filename = data_dir + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Predicting ' + image_filename)
            img = Image.open(image_filename)
            img = np.array(img)


            data = np.asarray(img_crop_translate(img, PATCH_SIZE, PATCH_SIZE, 0, 0))

            predictions_patch = model.predict_classes(data, verbose=1)

            img_prediction = label_to_img(img.shape[0], img.shape[1], 
                                          PATCH_SIZE, PATCH_SIZE, 
                                          predictions_patch)

            pimg = Image.fromarray((img_prediction*255.0).astype(np.uint8))
            pimg.save(pred_dir + "prediction_" + str(i) + ".png")

        else:
            print ('File ' + image_filename + ' does not exist')

In [26]:
train_cnn()

(10000, 16, 16, 3)
Train on 10000 samples, validate on 2500 samples
Epoch 1/3
10000/10000 [==============================] - 69s - loss: 0.6121 - fmeasure: 0.7100 - val_loss: 0.4747 - val_fmeasure: 0.8276
Epoch 2/3
10000/10000 [==============================] - 68s - loss: 0.5995 - fmeasure: 0.7125 - val_loss: 0.4803 - val_fmeasure: 0.8276
Epoch 3/3
10000/10000 [==============================] - 71s - loss: 0.5972 - fmeasure: 0.7125 - val_loss: 0.4909 - val_fmeasure: 0.8276
Test score: 0.490863571787
Test accuracy: 0.827599961281
Predicting post/images/pred_0.png
625/625 [==============================] - 1s     
Predicting post/images/pred_1.png
625/625 [==============================] - 1s     
Predicting post/images/pred_2.png
625/625 [==============================] - 1s     
Predicting post/images/pred_3.png
625/625 [==============================] - 1s     
Predicting post/images/pred_4.png
625/625 [==============================] - 1s     
Predicting post/images/pred_5.png
625/6

In [27]:
from keras.models import load_model
model = load_model('models/test.h5')
data_dir = 'post/images/'
pred_dir = 'post/prediction/'
for i in range(0, 19):
    imageid = "pred_%.1d" % i
    image_filename = data_dir + imageid + ".png"
    if os.path.isfile(image_filename):
        print ('Predicting ' + image_filename)
        img = Image.open(image_filename)
        img = np.array(img)

        print(img.shape)
        data = np.asarray(img_crop_translate(img, PATCH_SIZE, PATCH_SIZE, 0, 0))
        print(data.shape)

        predictions_patch = model.predict_classes(data, verbose=1)

        img_prediction = label_to_img(img.shape[0], img.shape[1], 
                                          PATCH_SIZE, PATCH_SIZE, 
                                          predictions_patch)

        pimg = Image.fromarray((img_prediction*255.0).astype(np.uint8))
        pimg.save(pred_dir + "prediction_" + str(i) + ".png")

    else:
        print ('File ' + image_filename + ' does not exist')

Predicting post/images/pred_0.png
(400, 400, 4)
(625, 16, 16, 3)
625/625 [==============================] - 1s     
Predicting post/images/pred_1.png
(400, 400, 4)
(625, 16, 16, 3)
625/625 [==============================] - 2s     
Predicting post/images/pred_2.png
(400, 400, 4)
(625, 16, 16, 3)
625/625 [==============================] - 1s     
Predicting post/images/pred_3.png
(400, 400, 4)
(625, 16, 16, 3)
512/625 [=======================>......] - ETA: 0s

KeyboardInterrupt: 